In [30]:
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.metrics import log_loss
from xgboost import XGBClassifier
import numpy as np
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split 
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn.utils import class_weight
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier 
import lightgbm as lgb
from sklearn import preprocessing


In [5]:
directory="D:\Purdue Courses\Fall Mod 1\MGMT 571 Data Mining using SAS Enterprise Miner\Project\\bankruptcy_Train"
directory_test="D:\Purdue Courses\Fall Mod 1\MGMT 571 Data Mining using SAS Enterprise Miner\Project\\bankruptcy_Test_X"

In [6]:
df_train=pd.read_csv(directory+"\\bankruptcy_Train.csv")
df_test=pd.read_csv(directory_test+'\\bankruptcy_Test_X.csv')

In [7]:
y=df_train['class']
X=df_train.drop(['class'],axis=1)

In [8]:
test_final= df_test.drop(['ID'],axis=1)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=1, stratify=y)

In [20]:
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X, y)

In [21]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(X_train)
x_test = sc.transform(X_test)

In [33]:
model = lgb.LGBMClassifier()

# fit the model with the training data
model.fit(X_res,y_res)

# predict the target on the train dataset
predict_train = model.predict(X_res)

In [34]:
predict_test = model.predict(test_final)

In [35]:
clf_cv_score = cross_val_score(model, X, y, cv=10, scoring='roc_auc')
print("=== Mean AUC Score ===")
print("Mean AUC Score - LGB: ", clf_cv_score.mean())

=== Mean AUC Score ===
Mean AUC Score - LGB:  0.9417749342754199


In [31]:
a=model.predict_proba(test_final)
np.savetxt(directory+"foo.csv", a, delimiter=",")

In [22]:
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
param_test ={'num_leaves': sp_randint(6, 50), 
             'min_child_samples': sp_randint(100, 500), 
             'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
             'subsample': sp_uniform(loc=0.2, scale=0.8), 
             'colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
             'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
             'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100]}

In [23]:
n_HP_points_to_test = 100

gs = RandomizedSearchCV(
    estimator=clf, param_distributions=param_test, 
    n_iter=n_HP_points_to_test,
    scoring='roc_auc',
    cv=3,
    refit=True,
    random_state=1,
    verbose=True)

clf = lgb.LGBMClassifier(max_depth=-1, random_state=1, 
                         silent=True, metric='None', 
                         n_jobs=4, n_estimators=5000)


In [24]:
clf.fit(X_res,y_res)

# predict the target on the train dataset
predict_train = clf.predict(X_test)

In [25]:
clf_cv_score = cross_val_score(clf, X, y, cv=10, scoring='roc_auc')
print("=== Mean AUC Score ===")
print("Mean AUC Score - LGB: ", clf_cv_score.mean())

=== Mean AUC Score ===
Mean AUC Score - LGB:  0.944563249245315


In [18]:
a=clf.predict_proba(test_final)
np.savetxt(directory+"foo.csv", a, delimiter=",")

In [27]:
clf.fit(X_res,y_res)

# predict the target on the train dataset
predict_train = clf.predict(test_final)

In [18]:
clf_cv_score = cross_val_score(clf, X, y, cv=10, scoring='roc_auc')
print("=== Mean AUC Score ===")
print("Mean AUC Score - LGB: ", clf_cv_score.mean())

=== Mean AUC Score ===
Mean AUC Score - LGB:  0.944563249245315
